# Evaluating Global Expainability

In [1]:
from utils import make_test_set_esnli, make_test_set_mnli, make_test_set_cose
from model import ZeroShotNLI, ZeroShotLearner
from explainer import Explainer

In [2]:
# number of explanations
NUM_EXPL = 10

#importing dataset
dataset_esnli = make_test_set_esnli(size=NUM_EXPL) # 9830 is dev size esnli (originally 9842 but 12 instance are wrongly annotated)
dataset_mnli = make_test_set_mnli(size=NUM_EXPL) # 9815 is dev size mnli
dataset_cose = make_test_set_cose(size=NUM_EXPL) # 718 (originally 1221 is dev size cose but 503 instances have bad explanations, i.e. the whole question highlighted))

Found cached dataset multi_nli (/Users/henningheyen/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset cos_e (/Users/henningheyen/.cache/huggingface/datasets/cos_e/v1.11/1.11.0/e8dc57a5b321a2a97063efb8d316d6d8a0d9a2d3a392dafc913e55bed42736d2)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
#calculating average explanation length per dataset
import numpy as np

full_dataset_esnli = make_test_set_esnli(size=9830, remove_bad_explanations=True) # 9830 is dev size esnli (originally 9842 but 12 instance are wrongly annotated)
full_dataset_cose = make_test_set_cose(size=718, remove_bad_explanations=True) # 718 (originally 1221 is dev size cose but 503 instances have bad explanations, i.e. the whole question highlighted))

len_esnli = [len(full_dataset_esnli['extractive_explanation'][i])/(len(full_dataset_esnli['sentence_pairs'][i][0].split())+len(full_dataset_esnli['sentence_pairs'][i][1].split())) for i in range(9830)]
len_cose = [len(full_dataset_cose['extractive_explanation'][i])/len(full_dataset_cose['question'][i].split()) for i in range(718)]

avg_len_esnli = np.mean(len_esnli)
avg_len_cose = np.mean(len_cose)

print('average explanation input ratio e-SNLI: ', avg_len_esnli)
print('average explanation input ratio CoS-e: ', avg_len_cose)

Found cached dataset cos_e (/Users/henningheyen/.cache/huggingface/datasets/cos_e/v1.11/1.11.0/e8dc57a5b321a2a97063efb8d316d6d8a0d9a2d3a392dafc913e55bed42736d2)


  0%|          | 0/2 [00:00<?, ?it/s]

average explanation input ratio e-SNLI:  0.19858092123245383
average explanation input ratio CoS-e:  0.2605110957380536


# Calculating Explanations

## Natural Language Inference

In [7]:
# Natural Language Inference
xsmall = ZeroShotNLI(model_name='nli-deberta-v3-xsmall')
small = ZeroShotNLI(model_name='nli-deberta-v3-small')
base = ZeroShotNLI(model_name='nli-deberta-v3-base')
large = ZeroShotNLI(model_name='nli-deberta-v3-large')

models = [
    xsmall,
    small,
    base,
    large
]

model_names = [
    'xsmall',
    'small',
    'base',
    'large'
]

In [8]:
explainer = Explainer(class_names=['contradiction', 'entailment', 'neutral'])

### MNLI

In [9]:
%%time

# Computing LIME explanations on xsmall
explanations_xsmall = explainer.compute_explanations(
    sentences = dataset_mnli['sentence_pairs'], 
    model=xsmall, 
    num_samples=100,  
    task='NLI',
    )

In [ ]:
%%time

# Computing LIME explanations on small
explanations_small = explainer.compute_explanations(
   sentences = dataset_mnli['sentence_pairs'], 
   model=small, 
   num_samples=100,  
   task='NLI',
   )

CPU times: user 7min 54s, sys: 8.85 s, total: 8min 3s
Wall time: 8min 6s


In [ ]:
%%time

# Computing LIME explanations on base
explanations_base = explainer.compute_explanations(
   sentences = dataset_mnli['sentence_pairs'], 
   model=base, 
   num_samples=100,  
   task='NLI',
   )

CPU times: user 16min 21s, sys: 25.7 s, total: 16min 47s
Wall time: 16min 57s


In [ ]:
%%time

# Computing LIME explanations on large
explanations_large = explainer.compute_explanations(
   sentences = dataset_mnli['sentence_pairs'], 
   model=large, 
   num_samples=100,  
   task='NLI',
   )

UsageError: Line magic function `%%time` not found.


In [ ]:
explanations_mnli = [
    explanations_xsmall, 
    explanations_small, 
    explanations_base, 
    explanations_large,
    ]

#### e-SNLI

In [ ]:
%%time

# Computing LIME explanations on xsmall
explanations_xsmall = explainer.compute_explanations(
    sentences = dataset_mnli['sentence_pairs'], 
    model=xsmall, 
    num_samples=100,  
    task='NLI',
    )

CPU times: user 3min 49s, sys: 3.68 s, total: 3min 53s
Wall time: 3min 52s


In [ ]:
%%time

# Computing LIME explanations on small
explanations_small = explainer.compute_explanations(
   sentences = dataset_mnli['sentence_pairs'], 
   model=small, 
   num_samples=100,  
   task='NLI',
   )

CPU times: user 7min 54s, sys: 8.85 s, total: 8min 3s
Wall time: 8min 6s


In [ ]:
%%time

# Computing LIME explanations on base
explanations_base = explainer.compute_explanations(
   sentences = dataset_mnli['sentence_pairs'], 
   model=base, 
   num_samples=100,  
   task='NLI',
   )

CPU times: user 16min 21s, sys: 25.7 s, total: 16min 47s
Wall time: 16min 57s


In [ ]:
%%time

# Computing LIME explanations on large
explanations_large = explainer.compute_explanations(
   sentences = dataset_mnli['sentence_pairs'], 
   model=large, 
   num_samples=100,  
   task='NLI',
   )

UsageError: Line magic function `%%time` not found.


In [ ]:
explanations_mnli = [
    explanations_xsmall, 
    explanations_small, 
    explanations_base, 
    explanations_large,
    ]

## Zero Shot Classification

In [ ]:
# Zero Shot Classification
xsmall = ZeroShotLearner(model_name='cross-encoder/nli-deberta-v3-xsmall')
small = ZeroShotLearner(model_name='cross-encoder/nli-deberta-v3-small')
base = ZeroShotLearner(model_name='cross-encoder/nli-deberta-v3-base')
large = ZeroShotLearner(model_name='cross-encoder/nli-deberta-v3-large')

models = [
    xsmall,
    small,
    base,
    large,
]

model_names = [
    'xsmall',
    'small',
    'base',
    'large',
]

In [ ]:
# no class names becuase of zero shot classification setting
explainer = Explainer()

In [ ]:
%%time

# Computing LIME explanations on xsmall
explanations_xsmall = explainer.compute_explanations(
   sentences = dataset_cose['question'], 
   model=model, 
   num_samples=100,  
   class_names_list=dataset_cose['candidate_labels_list']
   )

UsageError: Line magic function `%%time` not found.


In [ ]:
%%time

# Computing LIME explanations on small
explanations_small = explainer.compute_explanations(
   sentences = dataset_cose['question'], 
   model=model, 
   num_samples=100,  
   class_names_list=dataset_cose['candidate_labels_list']
   )

UsageError: Line magic function `%%time` not found.


In [ ]:
%%time

# Computing LIME explanations on base
explanations_base = explainer.compute_explanations(
   sentences = dataset_cose['question'], 
   model=model, 
   num_samples=100,  
   class_names_list=dataset_cose['candidate_labels_list']
   )

UsageError: Line magic function `%%time` not found.


In [ ]:
%%time

# Computing LIME explanations on large
explanations_large = explainer.compute_explanations(
   sentences = dataset_cose['question'], 
   model=model, 
   num_samples=100,  
   class_names_list=dataset_cose['candidate_labels_list']
   )

UsageError: Line magic function `%%time` not found.


In [ ]:
explanations_cose = [
    explanations_xsmall, 
    explanations_small, 
    explanations_base, 
    explanations_large,
    ]

## Faithfulness (Comprehensiveness and Sufficiency)

#### MNLI

In [ ]:
# Calculating aggregated comprehensiveness and sufficiency on 100 explanations
comp_list_mnli = []
suff_list_mnli = []

for i, model in enumerate(models):
    
    print('model: ', model_names[i])

    comp_agg = [explainer.aggregated_metric(metric='comprehensiveness', explanation=explanations_mnli[i][j], sentence=dataset_mnli['sentence_pairs'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='NLI') for j in range(NUM_EXPL)]

    comp_list_mnli.append(comp_agg)

    suff_agg = [explainer.aggregated_metric(metric='sufficiency', explanation=explanations_mnli[i][j], sentence=dataset_mnli['sentence_pairs'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='NLI') for j in range(NUM_EXPL)]

    suff_list_mnli.append(suff_agg)


In [ ]:
for i, model in enumerate(models):
    print(f'MNLI average aggregated comprehensiveness {model_names[i]}: ', np.mean(comp_list_mnli[i]))
    print(f'MNLI average aggregated sufficiency {model_names[i]}: ', np.mean(suff_list_mnli[i]))

#### e-SNLI

In [ ]:
# Calculating aggregated comprehensiveness and sufficiency on 100 explanations
comp_list_esnli = []
suff_list_esnli = []

for i, model in enumerate(models):
    
    print('model: ', model_names[i])

    comp_agg = [explainer.aggregated_metric(metric='comprehensiveness', explanation=explanations_mnli[i][j], sentence=dataset_esnli['sentence_pairs'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='NLI') for j in range(NUM_EXPL)]

    comp_list_esnli.append(comp_agg)

    suff_agg = [explainer.aggregated_metric(metric='sufficiency', explanation=explanations_mnli[i][j], sentence=dataset_esnli['sentence_pairs'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='NLI') for j in range(NUM_EXPL)]

    suff_list_esnli.append(suff_agg)

In [ ]:
for i, model in enumerate(models):
    print(f'e-SNLI average aggregated comprehensiveness {model_names[i]}: ', np.mean(comp_list_esnli[i]))
    print(f'e-SNLI average aggregated sufficiency {model_names[i]}: ', np.mean(suff_list_esnli[i]))

#### CoS-e

In [ ]:
# Calculating aggregated comprehensiveness and sufficiency on 100 explanations
comp_list_cose = []
suff_list_cose = []

for i, model in enumerate(models):
    
    print('model: ', model_names[i])

    comp_agg = [explainer.aggregated_metric(metric='comprehensiveness', explanation=explanations_mnli[i][j], sentence=dataset_cose['question'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='ZSC', candidate_labels=dataset_cose['candidate_labels_list'][j]) for j in range(NUM_EXPL)]

    comp_list_cose.append(comp_agg)

    suff_agg = [explainer.aggregated_metric(metric='sufficiency', explanation=explanations_mnli[i][j], sentence=dataset_cose['question'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='ZSC', candidate_labels=dataset_cose['candidate_labels_list'][j]) for j in range(NUM_EXPL)]

    suff_list_cose.append(suff_agg)

In [ ]:
for i, model in enumerate(models):
    print(f'CoS-e average aggregated comprehensiveness {model_names[i]}: ', np.mean(comp_list_cose[i]))
    print(f'CoS-e average aggregated sufficiency {model_names[i]}: ', np.mean(suff_list_cose[i]))

## Plausibility (IOU and Token Level F1 Scores)

#### e-SNLI

In [ ]:
explanation_tokens_lists = [explainer.get_explanation_list(explanations_esnli[i], top_percent=avg_len_esnli) for i in range(4)]
ground_truth_list = dataset_esnli['extractive_explanation']

iou_macro_scores = [explainer.compute_macro_iou(explanation_tokens_list, ground_truth_list) for explanation_tokens_list in explanation_tokens_lists]
tokenf1_macro_scores = [explainer.compute_macro_f1(explanation_tokens_list, ground_truth_list) for explanation_tokens_list in explanation_tokens_lists]

for i, model in enumerate(models):
    print(f'{model_names[i]} macro_iou for {NUM_EXPL} number of explanations (e-SNLI): ', iou_macro_scores[i])
    print(f'{model_names[i]} macro_f1 for {NUM_EXPL} number of explanations (e-SNLI): ', tokenf1_macro_scores[i])

#### CoS-e

In [ ]:
explanation_tokens_lists = [explainer.get_explanation_list(explanations_cose[i], top_percent=avg_len_cose) for i in range(4)]
ground_truth_list = dataset_cose['extractive_explanation']

iou_macro_scores = [explainer.compute_macro_iou(explanation_tokens_list, ground_truth_list) for explanation_tokens_list in explanation_tokens_lists]
tokenf1_macro_scores = [explainer.compute_macro_f1(explanation_tokens_list, ground_truth_list) for explanation_tokens_list in explanation_tokens_lists]

for i, model in enumerate(models):
    print(f'{model_names[i]} macro_iou for {NUM_EXPL} number of explanations (CoS-e): ', iou_macro_scores[i])
    print(f'{model_names[i]} macro_f1 for {NUM_EXPL} number of explanations (CoS-e): ', tokenf1_macro_scores[i])